In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_ta as ta
import seaborn as sns
import simplefix
from simplefix import FixMessage, FixParser
import socket
import ssl
import os

#### Credentials Setup

In [9]:
# Setup credentials for connection to cTrader
socket_buffer = 1024
server_url = "h35.p.ctrader.com"
server_port = 5211

sender = "demo.ctrader.3988979"
target = "CSERVER"
username = "3988979"
password = "SamaTrades123!"

#### Connection Setup

Create dynamic message sequence numbers that reset on login

In [10]:
global seq_num

In [11]:
def ctrader_connection():
    context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE

    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    ssock = context.wrap_socket(sock, server_hostname=server_url)

    try:
        ssock.connect((server_url, server_port))
        print(f"Connected to {server_url} on port {server_port}")
        return ssock
    except socket.error as socket_err:
        print(f"Socket error occurred: {socket_err}")
        raise
        
# Create the socket connection
active_socket = ctrader_connection()

Connected to h35.p.ctrader.com on port 5211


In [21]:
def send_ctrader_request(socket, fix_message):
    parser = simplefix.FixParser()
    seq_num = 1
    if fix_message.get(35 == "A"):
        fix_message.append_pair(34, seq_num, header=True)
        
    else:
        seq_num += 1
        fix_message.append_pair(34, seq_num, header=True)
        
    encoded_message = fix_message.encode()
        
    try:
        # Send the message bytes to the backend
        socket.send(encoded_message)
        # Fetch the API response
        response = socket.recv(socket_buffer)
        # Use the parser to decode the API response
        parser.append_buffer(response)
        # Parse the message to return
        return parser.get_message()
    except socket.error as socket_err:
        print(f"Socket error occurred: {socket_err}")
        raise
    except Exception as e:
        print(f"An error occurred: {e}")
        raise

#### Initiate a session to CTrader

In [18]:
# Create the FIX message
login_message = simplefix.FixMessage()
login_message.append_pair(8, "FIX.4.4")
login_message.append_pair(9, 126)
login_message.append_pair(35, "A")
login_message.append_pair(49, sender)
login_message.append_pair(56, target)
login_message.append_utc_timestamp(52)
login_message.append_pair(57, "QUOTE")
login_message.append_pair(98, 0)
login_message.append_pair(108, 0)
# Reset all sequence connections
login_message.append_pair(141, "Y")
login_message.append_pair(553,username)
login_message.append_pair(554,password)
login_message.append_pair(10, 131)

login_response_message = send_ctrader_request(active_socket, login_message)
if login_response_message.get(35).decode() == "A":
    print("Login Successful")
else:
    print(f"Login failed : {login_response_status.get(58)}")

8=FIX.4.4|9=126|35=A|49=demo.ctrader.3988979|56=CSERVER|52=20231211-19:27:51.008|57=QUOTE|98=0|108=0|141=Y|553=3988979|554=SamaTrades123!|10=131
Login Successful


#### Request Market Data

In [22]:
market_req_message = simplefix.FixMessage()
market_req_message.append_pair(8, "FIX.4.4")
market_req_message.append_pair(9, 135)
market_req_message.append_pair(35, "V")
market_req_message.append_pair(49, sender)
market_req_message.append_pair(56, target)
market_req_message.append_utc_timestamp(52)
market_req_message.append_pair(57, "QUOTE")
market_req_message.append_pair(262, "EURUSD")
market_req_message.append_pair(263, 1)
market_req_message.append_pair(264, 0)
market_req_message.append_pair(265, 1)
market_req_message.append_pair(267, 2)
market_req_message.append_pair(269, 0)
market_req_message.append_pair(269, 1)
market_req_message.append_pair(146, 1)
market_req_message.append_pair(55,1)
market_req_message.append_pair(10, 94)

market_data_response = send_ctrader_request(active_socket, market_req_message)
print("\n")
print(market_data_response)



8=FIX.4.4|9=658|35=X|34=4|49=CSERVER|50=QUOTE|52=20231211-19:27:53.176|56=demo.ctrader.3988979|268=14|279=0|269=1|278=3819140147|55=1|270=1.07655|271=1500000|279=0|269=1|278=3819140151|55=1|270=1.07654|271=100000|279=0|269=1|278=3819140149|55=1|270=1.07656|271=3000000|279=0|269=1|278=3819140148|55=1|270=1.07657|271=5000000|279=2|278=3819138912|55=1|279=2|278=3819138911|55=1|279=2|278=3819138910|55=1|279=2|278=3819138909|55=1|279=0|269=0|278=3819140144|55=1|270=1.07651|271=1000000|279=0|269=0|278=3819140143|55=1|270=1.0765|271=3000000|279=0|269=0|278=3819140142|55=1|270=1.07649|271=5000000|279=2|278=3819138906|55=1|279=2|278=3819138904|55=1|279=2|278=3819138908|55=1|10=157


#### Submit an order

In [28]:
# field=57|371=57|372=D|373=14|10=244
order_message = simplefix.FixMessage()
order_message.append_pair(8, "FIX.4.4")
order_message.append_pair(34, 12, header=True)
order_message.append_pair(54, 1)
order_message.append_pair(35, "D")
order_message.append_pair(141, "Y")
# Symbol
order_message.append_pair(55, "EURUSD")
order_message.append_pair(57, "QUOTE")
order_message.append_pair(49, sender)
order_message.append_pair(56, target)
order_message.append_utc_timestamp(52)
# Order Qty
order_message.append_pair(38, 100)
order_message.append_pair(40, 2)
# Price
order_message.append_pair(44, 0.01)
order_message.append_pair(10, 245)

order_submission_response = send_ctrader_request(active_socket, order_message)
print("\n")
print(order_submission_response)



8=FIX.4.4|9=550|35=X|34=9|49=CSERVER|50=QUOTE|52=20231211-19:27:56.717|56=demo.ctrader.3988979|268=11|279=0|269=1|278=3819147597|55=1|270=1.07655|271=3000000|279=0|269=1|278=3819147596|55=1|270=1.07654|271=1500000|279=2|278=3819146115|55=1|279=2|278=3819146116|55=1|279=0|269=0|278=3819147591|55=1|270=1.0765|271=1000000|279=0|269=0|278=3819147595|55=1|270=1.07651|271=600000|279=0|269=0|278=3819147594|55=1|270=1.07649|271=3000000|279=0|269=0|278=3819147593|55=1|270=1.07648|271=5000000|279=2|278=3819146113|55=1|279=2|278=3819146111|55=1|279=2|278=3819146109|55=1|10=217


#### Check order status

In [25]:
order_status_message = simplefix.FixMessage()
order_status_message.append_pair(8, "FIX.4.4")
order_status_message.append_pair(34, 12, header=True)
order_status_message.append_pair(35, "H")
order_status_message.append_pair(54, 1)

order_status_response = send_ctrader_request(active_socket, order_status_message)
print("\n")
print(order_status_response)



8=FIX.4.4|9=519|35=X|34=6|49=CSERVER|50=QUOTE|52=20231211-19:27:54.143|56=demo.ctrader.3988979|268=11|279=0|269=1|278=3819142347|55=1|270=1.07656|271=5000000|279=0|269=1|278=3819142345|55=1|270=1.07655|271=4000000|279=0|269=1|278=3819142344|55=1|270=1.07653|271=100000|279=0|269=1|278=3819142348|55=1|270=1.07654|271=500000|279=2|278=3819140147|55=1|279=2|278=3819140151|55=1|279=2|278=3819140149|55=1|279=2|278=3819140148|55=1|279=0|269=0|278=3819142340|55=1|270=1.0765|271=4000000|279=2|278=3819140144|55=1|279=2|278=3819140143|55=1|10=216
